In [ ]:
import polars as pl
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_PATH = "jane-street-real-time-market-data-forecasting"
data = pl.read_parquet(DATA_PATH + "/train.parquet")

Train Test Split Method
In training data, we preserve symbol_id from 0~30. And the date is from 0 to 1698.
In testing data, we use simply the last date 1699 for all symbol_id 0~39.

In [ ]:
train = data.filter((pl.col("symbol_id") < 31) & (pl.col("date_id") < 1698))

In [ ]:
test = data.filter(pl.col("date_id") == 1698)

In [ ]:
train = train.drop_nulls() 

In [ ]:
train_grouped = train.partition_by("symbol_id")

In [ ]:
feature = [col for col in train_pd.columns if col.startswith('feature')]
responder = 'responder_6'

In [ ]:
# Set parameters for LightGBM
params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting_type': 'gbdt',
        'learning_rate': 0.01,  # Lower learning rate
        'num_leaves': 31,
        'min_data_in_leaf': 20,  # Minimum number of data points in a leaf
        'feature_fraction': 0.8,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': -1
    }

In [ ]:
import lightgbm as lgb
models = []
predictions = []
true_values = []
for training_df in train_grouped:
    training_df = training_df.to_pandas()
    X_train = training_df[feature]
    y_train = training_df[responder]
    # Create LightGBM dataset
    train_data = lgb.Dataset(X_train, label=y_train)
    # Train the model
    model = lgb.train(params, train_data, num_boost_round=1000, valid_sets=[train_data])
    models.append((training_df["symbol_id"][0], model))
    print(training_df.shape)

In [ ]:
from sklearn.mixture import GaussianMixture

# Define the number of clusters
n_clusters = 2

# Initialize and fit the Gaussian Mixture Model
gmm = GaussianMixture(n_components=n_clusters, random_state=42).fit(X_train)


In [ ]:
n_clusters = 1
gmm_list = []
for training_df in train_grouped:
    X_train = training_df.to_pandas()[feature]
    # Predict the cluster for each data point
    gmm = GaussianMixture(n_components=n_clusters, random_state=42).fit(X_train)
    gmm_list.append((training_df["symbol_id"][0], gmm))

In [ ]:
import pandas as pd
predictions = pd.DataFrame(index=train_pd.index)
for i, model in models:
    predictions[f'prediction_{i}'] = model.predict(train_pd[feature])

In [ ]:
predictions["answer"] = train_pd[responder]
predictions["symbol_id"] = train_pd["symbol_id"]

In [ ]:
predictions_grouped = predictions.groupby("symbol_id")

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
lasso_list = []
lm_list = []
for dat in predictions_grouped:
    symbol_id = dat[0]
    dat = dat[1]
    X = dat.drop(columns=["answer", "symbol_id"])
    y = dat["answer"]
    clf = linear_model.Lasso(alpha=0.1).fit(X, y)
    reg = LinearRegression().fit(X, y)
    lasso_list.append((symbol_id, clf))
    lm_list.append((symbol_id, reg))

In [ ]:
prob_df = pd.DataFrame(index=test_pd.index)
for i, model in gmm_models:
    prob_df[f'prob_{i}'] = model.predict(test_pd[feature])

final_group = prob_df.idxmax(axis=1)
